<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/workflow_series_04_draw_a_workflow_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Draw a Workflow Graph to Help You Observe the Workflow`#Agently_Workflow_Showcase_Series - 04`

## Demo Description

**Author:** Agently Team

**Series Link**:

[Last Document]: [03 - Using Decorator to Create Chunks Faster](https://github.com/Maplemx/Agently/blob/main/playground/workflow_series_03_using_decorator_to_create_chunks.ipynb)

[Next Document]: [05 - Seperating Generation Steps and Reply to User in Every Step](https://github.com/Maplemx/Agently/blob/main/playground/workflow_series_05_seperating_generation_steps_and_reply_in_every_step.ipynb)

**Description:**

Now you have already learned how to use **Agently Workflow** to manage your LLMs application workflow. In this showcase, it is the one last trick.

"How can I check if the workflow I designed in code is correctly planned?" To have a graph to help your observation can be much better then reading the code, especially when there're too many chunks and connections.

No worried. **Agently Workflow** can generate mermaid code by using `workflow.draw()` method to help you observe your own workflow planning.

现在你已经学会如何使用**Agently Workflow**来管理你的LLMs应用的工作流程了，只剩下最后一个小技巧。

“我该怎么检查我在代码里实现的工作流逻辑是否正确？”尤其是当开发者在进行拥有大量流程块和连接关系的复杂工作流的时候，这个问题会更加突出。如果能够通过可视化图形的方式来帮助开发者来进行观察就好了。

不用担心，**Agently Workflow**可以通过`workflow.draw()`的方法来生成一段mermaid语法的代码，再通过mermaid渲染工具，就可以帮助你用可视化图形的方式来观察自己的工作流规划啦。

## Step 1: Install Packages

In [ ]:
!pip install -q -U Agently

> ℹ️ Agently Workflow is a new feature in version >= `3.2.0`

## Step 2: Using `workflow.draw()` to generate mermaid code and visualize it!

### Same Code in Showcase 3 without `workflow.start()`

In [9]:
import Agently
agent_factory = (
        Agently.AgentFactory()
        .set_settings("model.Google.auth.api_key", "")
        .set_settings("current_model", "Google")
)

# Step 0. Create Workflow Instance and Agent Instance
workflow = Agently.Workflow()
agent = agent_factory.create_agent()

# Step 1. Create Chunks
## Start Chunk
@workflow.chunk(
    chunk_id = "Start",
    type = "Start"
)

## User Input Chunk
@workflow.chunk(
    chunk_id = "User Input",
    handles = {
        "outputs": [{ "handle": "user_input" }],
    }
)
def user_input_executor(inputs, storage):
    return { "user_input": input("[User]: ") }

## Assistant Reply Chunk
@workflow.chunk(
    chunk_id = "Assistant Reply",
    handles = {
        "inputs": [{ "handle": "user_input" }],
        "outputs": [{ "handle": "assistant_reply" }],
    }
)
def assistant_reply_executor(inputs, storage):
    chat_history = storage.get("chat_history") or []
    reply = (
        agent
            .chat_history(chat_history)
            .input(inputs["user_input"])
            .start()
    )
    print("[Assistant]: ", reply)
    return { "assistant_reply": reply }

## Update Chat History Chunk
@workflow.chunk(
    chunk_id = "Update Chat History",
    handles = {
        "inputs": [
            { "handle": "user_input" },
            { "handle": "assistant_reply" },
        ],
    },
)
def update_chat_history_executor(inputs, storage):
    chat_history = storage.get("chat_history") or []
    chat_history.append({ "role": "user", "content": inputs["user_input"] })
    chat_history.append({ "role": "assistant", "content": inputs["assistant_reply"] })
    storage.set("chat_history", chat_history)
    return


@workflow.chunk(
    chunk_id = "Goodbye",
)
def goodbye_executor(inputs, storage):
    print("Bye~👋")
    return

# Step 2. Connect Chunks in Orders
workflow.chunks["Start"].connect_to(workflow.chunks["User Input"])
(
    workflow.chunks["User Input"].handle("user_input")
        .if_condition(lambda data: data == "#exit").connect_to(workflow.chunks["Goodbye"])
        .else_condition().connect_to(workflow.chunks["Assistant Reply"].handle("user_input"))
)
workflow.chunks["User Input"].handle("user_input").connect_to(workflow.chunks["Update Chat History"].handle("user_input"))
workflow.chunks["Assistant Reply"].handle("assistant_reply").connect_to(workflow.chunks["Update Chat History"].handle("assistant_reply"))
workflow.chunks["Update Chat History"].connect_to(workflow.chunks["User Input"])

### Now let's generate mermaid code and visuallize it!

In [ ]:
# We can use `mermaid-python` package to render mermaid code
# So install the package first
!pip install mermaid-python

In [11]:
# Check the mermaid code of the workflow first
mermaid_code = workflow.draw()
print("[Mermaid Code]: \n", mermaid_code)

[Mermaid Code]: 
 %%{ init: { 'flowchart': { 'curve': 'linear' }, 'theme': 'neutral' } }%%
%% Rendered By Agently %%
flowchart LR
    f9024ca5-18d9-44a8-9377-11f0e23c097a("Start") -.-> |"output -->-- input"| a5971148-6f84-488f-821a-89653b5a6941("User Input")
    a5971148-6f84-488f-821a-89653b5a6941("User Input") -.-> |"user_input -- ◇ -- input"| faf731fe-50f8-4074-adbf-7f3900812a68("Goodbye")
    a5971148-6f84-488f-821a-89653b5a6941("User Input") -.-> |"user_input -- ◇ -- user_input"| f3540386-0f51-47da-8fde-5ec8d6b21bc7("Assistant Reply")
    a5971148-6f84-488f-821a-89653b5a6941("User Input") -.-> |"user_input -->-- user_input"| 45500531-d427-4227-9cdd-e7b3f7f1aaaf("Update Chat History")
    f3540386-0f51-47da-8fde-5ec8d6b21bc7("Assistant Reply") -.-> |"assistant_reply -->-- assistant_reply"| 45500531-d427-4227-9cdd-e7b3f7f1aaaf("Update Chat History")
    45500531-d427-4227-9cdd-e7b3f7f1aaaf("Update Chat History") -.-> |"output -->-- input"| a5971148-6f84-488f-821a-89653b5a6941("User 

In [12]:
# Using `mermain-python` to render the code
from mermaid import Mermaid
Mermaid(mermaid_code)

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)